In [1]:
from airflow import DAG

from airflow.operators.python import PythonOperator


from time import strftime,localtime
from web_scraper import *

In [ ]:
with DAG('player_box_scores_etl',default_args={'retries': 4},description='ETL DAG tutorial',
        schedule_interval='0 10 * 10 *',catchup=False,tags=['nba_stats']) as dag:

        dag.doc_md = __doc__

        def extract(**kwargs):
                ti = kwargs['ti']

                bs = Box_scores()
                year = int(strftime("%Y"))

                start_reg = 9
                end_reg = 4
                start_post = end_reg
                end_post = 7
                reg_season = True

                month = int(strftime('%m'))
                year_range = str(year-1) + "-{:0>2d}".format(get_last_year(year))

                ti = kwargs['ti']
                if month <= end_reg or month >= start_reg:

                        url = bs.build_url(year_range, not reg_season)

                elif month >= start_post and month <= end_post:

                        url = bs.build_url(year_range, not reg_season)
                else:
                        return # exit dag
                
                return bs.iter_all(url,)

    


In [2]:
def get_last_year(year):
        if(year > 2000):
            return year % 2000
        return year % 100


In [ ]:
import pendulum

In [ ]:
d = pendulum.datetime(2022,5,27)

In [ ]:
link = 'https://www.nba.com/stats/players/boxscores/?Season=2021-22&SeasonType=Regular%20Season&DateFrom=05%2F01%2F2021&DateTo=05%2F27%2F2022'

In [9]:
import datetime
from datetime import timedelta

d = datetime.datetime

e = d.strftime(datetime.date.today(),'%m-%d-%Y').replace('-','%2F')

In [ ]:
s = link.find(e)
link[s:len(e)+s]

In [ ]:
e

In [3]:
def extract(**kwargs):
# ti = kwargs['ti']
        d = datetime.datetime
        from web_scraper import Box_scores
        bs = Box_scores()
        year = int(d.strftime(datetime.date.today(),"%Y"))
        
        s = datetime.date.today() - timedelta(days= 13)

        start_reg = 9
        end_reg = 4
        start_post = end_reg
        end_post = 7
        reg_season = True

        month = int(strftime('%m'))
        year_range = str(year-1) + "-{:0>2d}".format(get_last_year(year))

        # ti = kwargs['ti']
        if month <= end_reg or month >= start_reg:

                url = bs.build_url(year_range, not reg_season)

        elif month >= start_post and month <= end_post:

                url = bs.build_url(year_range,s, not reg_season)
        else:
                return # exit dag

        return bs.iter_all(url)


In [ ]:
pages = []
for i in extract():
    pages.append(i)

In [4]:
def get_player_and_team_ids(self,html):
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find("table")
    
    pids = []
    tids = []
    gids = []
    
    player_reg = '/player/\d+'
    team_reg = '/team/\d+'
    game_reg = '/game/\d+'

    for l in table.find_all('a'):
        player_match = search(player_reg,l.get('href'))

        team_match = search(team_reg,l.get('href'))
        
        game_match = search(game_reg,l.get('href'))

        if(player_match):
            player_id = int(search('\d+',player_match.group()).group())
            pids.append(player_id)
        elif(team_match):
            team_id = int(search('\d+',team_match.group()).group())
            tids.append(team_id)
        elif(game_match):
            game_id = search('\d+',game_match.group()).group()
            gids.append(game_id)
            
    return pids, tids, gids

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
df = pd.DataFrame()
for i in pages:
    df = pd.concat((df,pd.read_html(i)[0]))

In [ ]:
with open('shots.html','r') as f:
    h = f.read()

In [ ]:
soup = BeautifulSoup(h)

In [ ]:
soup.find('th')

In [ ]:
def build_url(self,year,s,reg_season = False):
    form = '%m-%d-%Y'
    d = datetime.datetime
    e = datetime.date.today() 

    s = d.strftime(s,form)
    e = d.strftime(e,form)
    start = '&DateFrom='+ s.replace('-','%2F')
    end = '&DateTo='+ e.replace('-','%2F')

In [5]:
from bs4 import BeautifulSoup
import re
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

from re import search
import pandas as pd


In [6]:
class Box_scores:
        def __init__(self):
            self.REG = "Regular%20Season"
            self.POST = "Playoffs"
            self.root = "https://www.nba.com/stats/players/boxscores/?" # + [teamname -prefix_1] +
            self.mid = "Season=" # +[year] YYYY-YY +
            self.tail = "&SeasonType=" # + [Regular%20Season|Playoffs]
            self.xpath = "/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[2]/div[1]/table"
            self.select_xpath = "/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select"
            
        def build_url(self,year,reg_season = False):
            if(reg_season):
                return self.root + self.mid + year + self.tail + self.REG
            
            return self.root + self.mid + year + self.tail + self.POST
        
        
        def iter_all(self, url):
            wait = 20
            with webdriver.Chrome() as driver:
                driver.get(url)
                element = WebDriverWait(driver,wait).until(EC.presence_of_element_located((By.XPATH,self.xpath)))
                s = driver.find_element(By.XPATH,self.select_xpath)
                t = s.text
                t = t.split("\n")

                
                for i in t[1:]:
                    s = driver.find_element(By.XPATH,self.select_xpath)
                    s = Select(s)
                    s.select_by_visible_text(i)
                    yield driver.page_source
                

            return
            
        
        def get_player_and_team_ids(self,html):
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find("table")
            
            pids = []
            tids = []
            gids = []
            
            player_reg = '/player/\d+'
            team_reg = '/team/\d+'
            game_reg = '/game/\d+'

            for l in table.find_all('a'):
                player_match = search(player_reg,l.get('href'))

                team_match = search(team_reg,l.get('href'))
                
                game_match = search(game_reg,l.get('href'))

                if(player_match):
                    player_id = int(search('\d+',player_match.group()).group())
                    pids.append(player_id)
                elif(team_match):
                    team_id = int(search('\d+',team_match.group()).group())
                    tids.append(team_id)
                elif(game_match):
                    game_id = search('\d+',game_match.group()).group()
                    gids.append(game_id)
                    
            return pids, tids, gids


In [10]:
def extract(year, regular):
    # ti = kwargs['ti']
    d = datetime.datetime
    bs = Box_scores()

    year_range = str(year-1) + "-{:0>2d}".format(get_last_year(year))

    url = bs.build_url(year_range, regular)

    return bs.iter_all(url)


In [13]:
for year in range(2013,2011,-1):
    # playoffs
    j = 0
    for i in extract(year, False):
        with open('../../../data/'+str(year)+'_playoffs_'+str(j)+'.html','w') as f:
            f.write(i)
        j+=1
    
    # j=0
    # for i in extract(year, True):
    #     with open('../../../data/'+str(year)+'_reg_season_'+str(j)+'.html','w') as f:
    #         f.write(i)
    #     j+=1


In [14]:
from re import search

In [15]:
def get_player_and_team_ids(html):
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find("table")
    
    pids = []
    tids = []
    gids = []
    
    player_reg = '/player/\d+'
    team_reg = '/team/\d+'
    game_reg = '/game/\d+'

    for l in table.find_all('a'):
        player_match = search(player_reg,l.get('href'))

        team_match = search(team_reg,l.get('href'))
        
        game_match = search(game_reg,l.get('href'))

        if(player_match):
            player_id = int(search('\d+',player_match.group()).group())
            pids.append(player_id)
        elif(team_match):
            team_id = int(search('\d+',team_match.group()).group())
            tids.append(team_id)
        elif(game_match):
            game_id = search('\d+',game_match.group()).group()
            gids.append(game_id)
            
    return pids, tids, gids


In [79]:
from os import listdir
listdir(fp)
ds

[]

In [110]:
fp = '../../../data/'
ds = listdir(fp)
j = 1 
length = len(ds)
for i in listdir(fp):
    with open(fp+i,'r') as f:
        html = f.read()
    pids, tids, gids = get_player_and_team_ids(html)

    df  = pd.read_html(html)[0]
    df['player_id'] = pids
    df['team_id'] = tids
    df['game_id'] = gids

    df = cast(df)

    players(df,engine)
    teams(df,engine)
    boxes(df,engine)
    
    print(j,'/',length)
    j+=1

1 / 5882


In [108]:
def cast(df):
    df.drop(columns=['Season','FP','FG%','FT%','3P%','REB'],inplace = True)

    di = {'3pm':'pm3', '3pa':'pa3','match\xa0up':'match_up','min':'mins',
    'w/l': 'result','+/-':'plus_minus','game\xa0date':'game_day'}
    # dicto = dict(df.columns,)

    df.rename(str.lower,axis='columns',inplace=True)

    df.rename(columns=di,inplace =True)

    df['game_day'] = pd.to_datetime(df['game_day'])
    str_cols = ['player','team','match_up','result','game_id']
    dt = ['string'] * len(str_cols)
    df = df.astype(dict(zip(str_cols,dt)))

    return df 


In [62]:
from sqlalchemy import create_engine
from os import environ
engine = create_engine("mariadb+mariadbconnector://"\
                        +environ.get("USER")+":"\
                        +environ.get("PSWD")+\
                        "@127.0.0.1:3306/nba")

In [63]:
fr = pd.read_sql('select * from box_scores;',engine)
# df = df[fr.columns]

In [102]:
def players(df,engine):
    d = df[['player_id','player']]
    d = d.rename(columns={'player':'name','player_id':'id'})
    d = d.drop_duplicates('id')

    d.astype({'id':'int64','name':'string'})

    players = pd.read_sql('select id from players',engine)
    d = d[~d['id'].isin(players.values[:,0])]#players.values[:,0]

    d.to_sql('players',engine,if_exists='append',index=False)


    return 

In [88]:
players  = pd.read_sql('select id from players',engine)

In [100]:
df[~(df['player_id'].isin(players.values[:,0]))].head(8)
# 101108

,player,team,match_up,game_date,result,mins,pts,fgm,fga,pm3,...,dreb,ast,stl,blk,tov,pf,plus_minus,player_id,team_id,game_id
0,Harrison Barnes,SAC,SAC vs. POR,2019-11-12,W,32,16,4,8,1,...,3,2,1,0,3,1,2,203084,1610612758,0021900153
1,Damian Lillard,POR,POR @ SAC,2019-11-12,L,38,27,6,10,2,...,4,5,2,1,5,1,-11,203081,1610612757,0021900153
2,Mario Hezonja,POR,POR @ SAC,2019-11-12,L,22,2,1,4,0,...,2,2,0,0,2,1,-1,1626209,1610612757,0021900153
3,Nemanja Bjelica,SAC,SAC vs. POR,2019-11-12,W,36,19,8,14,3,...,9,4,0,0,0,4,4,202357,1610612758,0021900153
4,Devon Hall,OKC,OKC @ IND,2019-11-12,L,7,2,1,1,0,...,0,0,0,0,1,1,5,1628985,1610612760,0021900146
5,Abdel Nader,OKC,OKC @ IND,2019-11-12,L,20,6,2,9,0,...,2,2,0,1,3,1,-11,1627846,1610612760,0021900146
6,Nerlens Noel,OKC,OKC @ IND,2019-11-12,L,21,2,1,4,0,...,1,2,2,3,1,3,-16,203457,1610612760,0021900146
8,Domantas Sabonis,IND,IND vs. OKC,2019-11-12,W,29,18,9,19,0,...,9,2,0,1,2,3,27,1627734,1610612754,0021900146


In [103]:
def teams(df,engine):
    d = df[['team_id','team']]
    d = d.rename(columns={'team':'name','team_id':'id'})
    d = d.drop_duplicates('id')

    d.astype({'id':'int64','name':'string'})
    
    teams = pd.read_sql('select id from teams',engine)

    d = d[~d['id'].isin(teams.values[:,0])]

    d.to_sql('teams',engine,if_exists='append',index=False)


    return 

In [55]:
teams(df,engine)

In [85]:
def boxes(df, engine):
    cols = ['player_id', 'team_id', 'game_id', 'mins', 'pts', 'fgm', 'fga', 'pm3',
        'pa3', 'ftm', 'fta', 'oreb', 'dreb', 'ast', 'stl', 'blk', 'tov', 'pf',
        'plus_minus', 'result', 'game_day', 'match_up']

    df = df[cols]

    df.to_sql('box_scores',engine,if_exists='append',index=False)

In [69]:
boxes(df, engine)